In [95]:
import math, numpy, random

## Model structure

In [91]:
# Class to add linked list 'memory' to a mathematical expression
# Added backward functions + additional expressions
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self._backward = lambda: None
        self.data = data
        self.grad = 0.0 # zeroing gradients, seen at the start of any training loop
        self._prev = set(_children)
        self._op = _op
        self.label = label

    def __repr__(self):
        return f'Value(data={self.data})'

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward

        return out

    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)

    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out =  Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out

    def __rmul__(self, other):
        return self * other

    def __truediv__(self, other):
        return self * other ** -1

    def tanh(self):
        n = self.data
        t = (math.exp(2*n)-1)/(math.exp(2*n) + 1)
        out = Value(t, (self,), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        out._backward = _backward
        
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "Only int/floats"
        out = Value(self.data**other, (self,), f'**{other}')

        def _backward():
            self.grad += out.grad * other * self.data ** (other - 1)
        out._backward = _backward

        return out

    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out


    def backward(self):
        # build topological graph
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [223]:
class Module:
    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0

    def parameters(self):
        return []

In [224]:
class Neuron(Module):
    def __init__(self, num_in):
        self.w = [Value(random.uniform(-1,1)) for _ in range(num_in)]
        self.b = Value(random.uniform(-1,1))

    def __call__(self, x):
        # 2 * x + b [dot product[matrix]]
        act = sum((wi * xi for wi, xi in zip(self.w, x)), self.b)
        out = act.tanh()
        return out

    def parameters(self):
        return self.w + [self.b]

In [225]:
class Layer(Module):
    def __init__(self, num_in, num_out):
        self.neurons = [Neuron(num_in) for _ in range(num_out)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

In [226]:
class MLP(Module):
    def __init__(self, num_in, num_out):
        z = [num_in] + num_out
        self.layers = [Layer(z[i], z[i+1]) for i in range(len(num_out))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

## Gradient Descent (Training)

In [233]:
# hyperparameters
lr = 0.05
epochs = 50

In [227]:
x = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
y = [1.0, -1.0, -1.0, 1.0]

In [234]:
model = MLP(3, [4, 4, 1])
ypred = [model(x) for x in x]
len(model.parameters())

41

In [235]:
for epoch in range(epochs):
    ypred = [model(x) for x in x]
    
    loss = sum((yout - ygt)**2 for ygt, yout in zip(y, ypred))

    model.zero_grad()
    
    loss.backward()

    for p in model.parameters():
        p.data += -lr * p.grad

    if epoch % 10 == 0: print(epoch, loss.data)
ypred

0 5.913512266486831
10 0.14686771653261432
20 0.052228039924839056
30 0.03047135243423118
40 0.02117122573067071


[Value(data=0.9668943619296837),
 Value(data=-0.9361564879286824),
 Value(data=-0.9271743911509556),
 Value(data=0.9224518354679637)]

## Generation

In [236]:
model(x[0])

Value(data=0.9673113905022773)